In [80]:
Book = dict()

ticker = ['KISKGATT', 'KISKCALL', 'LEGATRUU', 'IBXXSIG1', 'JPEIGLBL', 'LF89TRUU', 'MXKR000V' , 'MXKR000G','KSDYKOSP', 
'SPKRDOKP','KOSPI2', 'MXWD', 'MXEF',  'DJUSRE','IQHGMS']

indexTicker =  [ticker[i] + ' Index' for i in range(len(ticker))]
fields = ['last_price']

Book['Tickers'] = indexTicker
Book['Flds'] = fields

In [76]:
import blpapi
import numpy as np
import pandas as pd
import xlwings
import openpyxl 
from xbbg import blp
from datetime import datetime, timedelta
from copy import deepcopy


"""
------------------------------
업데이트 필요 사항: 

(1) xlsxPath가 없는 경우(초기파일이 없는 경우)를 가정해서 만들어야함
(2) Columns 이거 flds가 다양할 때 문제 발생시킬 듯
(3) 기존 엑셀 파일에서 함수 오류로 인해서 첫번째 date값 #NAME? 뜨는 문제 컨트롤 필요(수동으로 할 것인가)
(4) metadata를 유지할 것인가. 유지할 계획이면 update 시 값 수정 해줘야함. 인덱싱도 바뀌어야함.
------------------------------
"""

startDate = '1999-12-31'
endDate = datetime.now().strftime('%Y-%m-%d')
startUpdate = dict() # Sheet별로 lastUpdate가 다를 가능성

xlsxPath = "./test.xlsx" # test용 임시파일 경로
Tickers = ['SPX Index', 'KOSPI']
Flds = ['last_price']

"""
Tickers = Book['Tickers']
Flds = Book['Flds']
"""

Columns = deepcopy(Flds) # 기존 파일은 일자 외에 기본 metadate가 포함되어 있어서 Note라 명칭
Columns.insert(0,'Note')

rawSheet = pd.read_excel(xlsxPath, sheet_name=Tickers, engine='openpyxl', header=None)
dfData = dict()

startPoint = 6 # 엑셀 시트에서 값이 시작하는 번호 (예시: 6)

for i in range(len(Tickers)):
    dfData[Tickers[i]] = rawSheet[Tickers[i]].iloc[startPoint:]
    rawSheet[Tickers[i]] = rawSheet[Tickers[i]].iloc[:startPoint]

    dfData[Tickers[i]].columns = Columns
    dfData[Tickers[i]].index = dfData[Tickers[i]]['Note']
    dfData[Tickers[i]] = dfData[Tickers[i]].drop(labels='Note', axis=1)
    dfData[Tickers[i]].index = [datetime.strftime(k, '%Y-%m-%d') for k in dfData[Tickers[i]].index]
    
for i in range(len(Tickers)):
    startUpdate[Tickers[i]] = datetime.strptime(dfData[Tickers[i]].index[-1], '%Y-%m-%d')+timedelta(days=1) # 마지막 날짜 인덱싱

#rawSheet['KOSPI'].head()
dfData['KOSPI'].head()


,last_price
1999-12-31,25
2000-01-01,26
2000-01-02,27
2000-01-03,28
2000-01-04,29


In [ ]:
"""
------------------------------
업데이트 필요 사항: 

(1) bdh 함수의 return이 pandas DataFrame인지 확인 필요
(2) kwargs 잘 들어가는지 확인 필요
(3) ticker 혹은 flds의 문제가 발생할 경우의 에러처리 필요
------------------------------
"""

def parser(ticker, fldList, startDate, endDate, **kwargs):
    tmpData = blp.bdh(tickers=ticker, flds=fldList, start_date=startDate, end_date=endDate, kwargs=kwargs)

    if type(tmpData) == pd.DataFrame:
        tmpData.index = [datetime.strftime(k, '%Y-%m-%d') for k in tmpData.index] # index datetime 객체화
        tmpData.columns = fldList
    else:
        print('ERROR in Sheet {sheetName}! : Check your ticker, flds'.format(sheetName=ticker))
        tmpData = None

    return tmpData

In [ ]:
updateData = dict()

for ticker in Tickers:
    updateData[ticker] = parser(ticker=ticker, fldList=Flds, startDate=datetime.strftime(startUpdate[ticker], '%Y-%m-%d'), endDate=endDate)
    dfData[ticker] = pd.concat([dfData[ticker], updateData[ticker]], axis=0)
    
    dfData[ticker].reset_index(inplace=True)
    rawSheet[ticker] = pd.concat([rawSheet[ticker], dfData[ticker]], axis=0)

    rawSheet[ticker].to_excel(xlsxPath, sheet_name=ticker, index=False)